In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import numpy as np

import requests
from bs4 import BeautifulSoup
import re
import time
import os

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False


## 최근 변경내역 페이지 정보 수집하기

In [2]:
# 주소
site = 'https://namu.wiki/RecentChanges'

In [3]:
# 요청한다.
res = requests.get(site)
html = res.text
soup = BeautifulSoup(html, 'lxml')
soup

<!DOCTYPE html>
<html data-n-head-ssr=""><head><title>최근 변경내역 - 나무위키</title><meta charset="utf-8" data-n-head="ssr"/><meta content="user-scalable=no, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0, width=device-width" data-n-head="ssr" name="viewport"/><meta content="ie=edge" data-n-head="ssr" http-equv="x-ua-compatible"/><meta content="the seed" data-n-head="ssr" name="generator"/><meta content="yes" data-n-head="ssr" name="mobile-web-app-capable"/><meta content="나무위키" data-n-head="ssr" name="application-name"/><meta content="나무위키" data-n-head="ssr" name="msapplication-tooltip"/><meta content="/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8" data-n-head="ssr" name="msapplication-starturl"/><meta content="#008275" data-n-head="ssr" name="theme-color"/><meta content="noarchive" data-n-head="ssr" name="googlebot"/><link data-n-head="ssr" href="https://namu.wiki/RecentChanges" rel="canonical"/><link data-n-head="ssr" href="/opensearch.xml" rel="search" title="나무위키" typ

In [4]:
# 데이터를 수집한다.
table_tag = soup.select_one('#app > div > div:nth-child(2) > article > div > table')
tbody_tag = table_tag.select_one('tbody')
tr_tag_list = tbody_tag.select('tr')
tr_tag_list

[<tr data-v-3e749e98=""><td data-v-3e749e98=""><a data-v-3e749e98="" href="/w/%EB%A7%88%EC%9D%B4%EC%95%BC(%EB%9E%91%EA%B7%B8%EB%A6%BF%EC%82%AC%20RE:%EC%9D%B8%EC%B9%B4%EB%84%A4%EC%9D%B4%EC%85%98)">마이야(랑그릿사 RE:인카네이션)</a> <a data-v-3e749e98="" href="/history/%EB%A7%88%EC%9D%B4%EC%95%BC(%EB%9E%91%EA%B7%B8%EB%A6%BF%EC%82%AC%20RE:%EC%9D%B8%EC%B9%B4%EB%84%A4%EC%9D%B4%EC%85%98)">[역사]</a> <a data-v-3e749e98="" href="/diff/%EB%A7%88%EC%9D%B4%EC%95%BC(%EB%9E%91%EA%B7%B8%EB%A6%BF%EC%82%AC%20RE:%EC%9D%B8%EC%B9%B4%EB%84%A4%EC%9D%B4%EC%85%98)?rev=4&amp;oldrev=3">[비교]</a> <a data-v-3e749e98="" href="/discuss/%EB%A7%88%EC%9D%B4%EC%95%BC(%EB%9E%91%EA%B7%B8%EB%A6%BF%EC%82%AC%20RE:%EC%9D%B8%EC%B9%B4%EB%84%A4%EC%9D%B4%EC%85%98)">[토론]</a> <span data-v-3e749e98="">(<span class="p" data-v-3e749e98="" data-v-3ee54395="">+1004</span>)</span></td> <td data-v-3e749e98=""><div class="v-popover" data-v-3e749e98="" data-v-e473dc5a=""><div aria-describedby="popover_gplw1muu8n" class="trigger" style="display:inline-bl

In [5]:
# 링크 주소들을 담을 리스트
link_list = []

# 위에서 수집한 tr 태그의 수만큼 반복한다.
for tr_tag in tr_tag_list :
    # tr태그 내에서 td 태그들을 가져온다.
    td_list = tr_tag.select('td')
    # 추출한 td 태그중 한 줄만 사용한다.
    td_tag = td_list[0]
    # td 태그 내의 a 태그를 가져온다.
    a_tag_list = td_tag.select('a')
    # 가져온 A 태그가 있을경우에만 작업을 한다.
    if len(a_tag_list) > 0:
        # 첫번째 a 태그의 href 속성값을 가져온다.
        href = a_tag_list[0].get('href')
        # 리스트에 담는다.
        link_list.append(href)
print(link_list)

['/w/%EB%A7%88%EC%9D%B4%EC%95%BC(%EB%9E%91%EA%B7%B8%EB%A6%BF%EC%82%AC%20RE:%EC%9D%B8%EC%B9%B4%EB%84%A4%EC%9D%B4%EC%85%98)', '/w/%EC%BC%84%EC%8B%9C%EB%A1%9C', '/w/%ED%81%AC%EB%A3%A8%EC%97%98%EB%9D%BC%20%EB%93%9C%20%EB%B9%8C', '/w/%EB%B0%A5(%EC%B2%A0%EA%B6%8C)', '/w/%EB%AA%A8%EB%8B%88%ED%84%B0%20%EC%95%94', '/w/Bryn/%EB%94%94%EC%8A%A4%EC%BD%94%EA%B7%B8%EB%9E%98%ED%94%BC', '/w/%ED%8C%8C%EC%9D%BC:JHu4nVk.jpg', '/w/%EA%B5%AC%EC%8A%AC%EC%95%84%EC%9D%B4%EC%8A%A4%EB%A7%9B%20%EC%BF%A0%ED%82%A4', '/w/%EB%A1%9C%EC%8A%A4%EC%95%A4%EC%A0%A4%EB%A0%88%EC%8A%A4%20%ED%81%B4%EB%A6%AC%ED%8D%BC%EC%8A%A4', '/w/%EC%A7%80%EA%B5%AC%EB%A9%B8%EB%A7%9D', '/w/%EA%B8%B0%EC%B9%A8', '/w/%EC%8A%A4%ED%8B%B0%EB%B8%8C%20%EC%96%B4%EC%9C%88', '/w/%ED%98%84%EB%8C%80%20%EC%97%91%EC%84%BC%ED%8A%B8', '/w/GS%EC%B9%BC%ED%85%8D%EC%8A%A4%20%EC%84%9C%EC%9A%B8%20KIXX/2020-21%EC%8B%9C%EC%A6%8C', '/w/%ED%81%AC%EB%A3%A8%EC%97%98%EB%9D%BC%20%EB%93%9C%20%EB%B9%8C', '/w/%ED%83%90%EC%9A%95%EB%A7%88%EC%99%95', '/w/%ED%98%84%EB%8C%80%20%EC%9

In [6]:
# 주소의 수 만큼 반복한다
for sub_url in link_list :

    time.sleep(1)
    
    # 데이터를 담을 딕셔너리
    data_dict = {
        'title' : [],
        'content' : []
    }
    # 페이지 내의 데이터를 수집한다.
    # 요청 주소
    site2 = f'https://namu.wiki{sub_url}'
    print(f'{site2} 수집중')

    # 요청한다.
    res2 = requests.get(site2)
    soup2 = BeautifulSoup(res2.text, 'lxml')
    # 글 전체를 가져온다.
    article_tag = soup2.select_one('#app > div > div:nth-child(2) > article')
    # 제목을 가져온다.
    h1_tag = article_tag.select_one('h1')
    # 글을 가지고 있는 태그들을 가져온다.
    content_paragraphs = article_tag.select('div.wiki-paragraph')
    # 글의 수만큼 반복한다.
    # 글 내용을 하나의 문자열로 합친다.
    content_text = ''
    for content_tag in content_paragraphs :
        content_text = content_text + content_tag.text

    # 제거하고자 하는 문자열들을 제거한다.
#     content_text = content_text.replace('','')
    content_text = content_text.replace('번 문단', '')
    content_text = content_text.replace('문서의 r 판,', '')
    content_text = content_text.replace('(이전 역사)', '')
    content_text = content_text.strip()
    

    # 글 제목
    title_text = h1_tag.text.strip()

#     print(title_text)
#     print(content_text)
#     print('-------------------------------------------------------------')
    # 데이터를 담는다
    data_dict['title'].append(title_text)
    data_dict['content'].append(content_text)
    
        # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    if os.path.exists('wiki.csv') == False :
        # 파일이 없을 경우
        df1.to_csv('wiki.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('wiki.csv', encoding='utf-8-sig', index=False, header=False, mode='a')

https://namu.wiki/w/%EB%A7%88%EC%9D%B4%EC%95%BC(%EB%9E%91%EA%B7%B8%EB%A6%BF%EC%82%AC%20RE:%EC%9D%B8%EC%B9%B4%EB%84%A4%EC%9D%B4%EC%85%98) 수집중
https://namu.wiki/w/%EC%BC%84%EC%8B%9C%EB%A1%9C 수집중
https://namu.wiki/w/%ED%81%AC%EB%A3%A8%EC%97%98%EB%9D%BC%20%EB%93%9C%20%EB%B9%8C 수집중
https://namu.wiki/w/%EB%B0%A5(%EC%B2%A0%EA%B6%8C) 수집중
https://namu.wiki/w/%EB%AA%A8%EB%8B%88%ED%84%B0%20%EC%95%94 수집중
https://namu.wiki/w/Bryn/%EB%94%94%EC%8A%A4%EC%BD%94%EA%B7%B8%EB%9E%98%ED%94%BC 수집중
https://namu.wiki/w/%ED%8C%8C%EC%9D%BC:JHu4nVk.jpg 수집중
https://namu.wiki/w/%EA%B5%AC%EC%8A%AC%EC%95%84%EC%9D%B4%EC%8A%A4%EB%A7%9B%20%EC%BF%A0%ED%82%A4 수집중
https://namu.wiki/w/%EB%A1%9C%EC%8A%A4%EC%95%A4%EC%A0%A4%EB%A0%88%EC%8A%A4%20%ED%81%B4%EB%A6%AC%ED%8D%BC%EC%8A%A4 수집중
https://namu.wiki/w/%EC%A7%80%EA%B5%AC%EB%A9%B8%EB%A7%9D 수집중
https://namu.wiki/w/%EA%B8%B0%EC%B9%A8 수집중
https://namu.wiki/w/%EC%8A%A4%ED%8B%B0%EB%B8%8C%20%EC%96%B4%EC%9C%88 수집중
https://namu.wiki/w/%ED%98%84%EB%8C%80%20%EC%97%91%EC%84%BC%ED%8A%B8 수

In [7]:
wiki_df = pd.read_csv('wiki.csv')
wiki_df


,title,content
0,마이야(랑그릿사 RE:인카네이션),랑그릿사 모바일에서 아레스와 같이 이벤트 뽑기를 통해 출시되었다 성우는 우치야마 유...
1,켄시로,은(는) 여기로 연결됩니다. 다른 뜻에 대한 내용은 아래 문서를 참조하십시오.일본...
2,크루엘라 드 빌,[ 펼치기 · 접기 ]블랙 피트그림하일드트리메인 부인퀸 오브 하트후크 선장말레피센트...
3,밥(철권),은(는) 여기로 연결됩니다. 밥(철권)이 사용하는 기술의 커맨드에 대한 내용은 밥(...
4,모니터 암,컴퓨터 주변기기 [ 펼치기 · 접기 ]모니터키보드마우스마이크스피커헤드폰모니터키보드마...
...,...,...
93,박명수/별명,"상위 문서: 박명수참고할 만한 문서 : 김태균/별명, 박용택/별명, 정준하/별명, ..."
94,바나나맛 쿠키,쿠키런 시리즈의 등장 쿠키들 [ 펼치기 · 접기 ] [ ㄱ ~ ㄹ ]감초맛경찰맛고블...
95,Rollin'(브레이브걸스)/역주행,은(는) 여기로 연결됩니다. 상위 문서인 롤린에 대한 내용은 Rollin'(브레이브...
96,Public Enemy,1. 미국의 힙합 그룹2. 공공의 적을 뜻하는 영어 단어3. 영화로큰롤 명예의 전당...
